<a href="https://colab.research.google.com/github/rohmadyudiutomo/p300/blob/main/Deepchem_solubility%20use%20own%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dPlease click below to open this notebook with colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r3QAoLsI-k6se1EubeepUs8p0Bqvapb_?usp=sharing)

The Deepchem and dataset setup below was taken from the official tutorial: [link ](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/03_Modeling_Solubility.ipynb)

In [1]:
# Installing conda
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  16540      0 --:--:-- --:--:-- --:--:-- 16540


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
# Installing Deepchem
!pip install --pre deepchem
import deepchem
deepchem.__version__

     |████████████████████████████████| 409kB 13.6MB/s 
  Created wheel for deepchem: filename=deepchem-2.4.0rc1.dev20201118005902-cp36-none-any.whl size=516160 sha256=0045be64ec97d7b5b0194117c23d5724af88ddc5247d74e735aeda85f1bb24ec
  Stored in directory: /root/.cache/pip/wheels/d0/62/46/62d0ff22abaf0778aa1ef94cde4282cb45fcb190f5cdd4ec75
Successfully built deepchem


'2.4.0-rc1.dev'

In [16]:
# Getting the delaney dataset
!wget https://raw.githubusercontent.com/rohmadyudiutomo/p300/main/delaney-processed.csv
from deepchem.utils.save import load_from_disk
dataset_file= "delaney-processed.csv"

# Loading the data from the CSV file
loader = deepchem.data.CSVLoader(tasks=["measured log solubility in mols per litre"], 
                                 smiles_field="smiles", 
                                 featurizer=deepchem.feat.ConvMolFeaturizer())
# Featurizing the dataset with ConvMolFeaturizer
dataset = loader.featurize(dataset_file)

--2020-11-18 01:08:11--  https://raw.githubusercontent.com/rohmadyudiutomo/p300/main/delaney-processed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96699 (94K) [text/plain]
Saving to: ‘delaney-processed.csv.1’

delaney-processed.c 100%[===================>]  94.43K  --.-KB/s    in 0.005s  

2020-11-18 01:08:11 (18.1 MB/s) - ‘delaney-processed.csv.1’ saved [96699/96699]



smiles_field is deprecated and will be removed in a future version of DeepChem.Use feature_field instead.
/usr/local/lib/python3.6/dist-packages/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


In [17]:
    # Splitter splits the dataset 
    # In this case it's is an equivalent of train_test_split from sklearn
    splitter = deepchem.splits.RandomSplitter()
    # frac_test is 0.01 because we only use a train and valid as an example
    train, valid, _ = splitter.train_valid_test_split(dataset,
                                                      frac_train=0.7,
                                                      frac_valid=0.29,
                                                      frac_test=0.01)
    # Normalizer will normalize y values in the dataset
    normalizer = deepchem.trans.NormalizationTransformer(transform_y=True, 
                                                         dataset=train, 
                                                         move_mean=True)
    train = normalizer.transform(train)
    test = normalizer.transform(valid)

In [18]:
print(f"Size of the training data: {len(train.ids)}")
print(f"Size of the validation data: {len(valid.ids)}")
print(test)

Size of the training data: 789
Size of the validation data: 327
<DiskDataset X.shape: (327,), y.shape: (327, 1), w.shape: (327, 1), ids: ['c1ccoc1' 'Nc2nc1[nH]cnc1c(=O)[nH]2 ' 'Oc1cccc(c1)N(=O)=O' ...
 'COP(=O)(NC(C)=O)SC ' 'Oc1ccc(cc1)c2ccccc2' 'ClCc1ccccc1'], task_names: ['measured log solubility in mols per litre']>


In [19]:
# GraphConvModel is a GNN model based on 
# Duvenaud, David K., et al. "Convolutional networks on graphs for
# learning molecular fingerprints."
from deepchem.models import GraphConvModel
graph_conv = GraphConvModel(1,
                            batch_size=50,
                            mode="regression")
# Defining metric. Closer to 1 is better
metric = deepchem.metrics.Metric(deepchem.metrics.pearson_r2_score)


In [20]:
# Fitting the model
graph_conv.fit(train, nb_epoch=20)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.10651574134826661

In [21]:
# Reversing the transformation and getting the metric scores on 2 datasets
train_scores = graph_conv.evaluate(train, [metric], [normalizer])
valid_scores = graph_conv.evaluate(valid, [metric], [normalizer])
print(f"Train Scores: {train_scores}")
print(f"Validation Scores: {valid_scores}")

Train Scores: {'pearson_r2_score': 0.8335385280554164}
Validation Scores: {'pearson_r2_score': 0.7014367920584546}


In [22]:
smiles = ['O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(OC)=C(O)C=C2',
'O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2',
'O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(Br)=C(O)C=C2',
'O=C(CC(/C=C/C1=CC=C(O)C(OC)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2',
'O=C(CC(/C=C/C1=CC=C(O)C(OC)=C1)=O)/C=C/C2=CC(Br)=C(O)C=C2',
'O=C(CC(/C=C/C1=CC=C(O)C(Br)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2',
'O=C(/C(CCC/1)=C/C2=CC(OC)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3',
'O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3',
'O=C(/C(CCC/1)=C/C2=CC(Br)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3',
'O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(OC)=C3',
'O=C(/C(CCC/1)=C/C2=CC(Br)=C(O)C=C2)C1=C\C3=CC=C(O)C(OC)=C3',
'O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(Br)=C3']

In [23]:
from rdkit import Chem
mols = [Chem.MolFromSmiles(s) for s in smiles]
featurizer = deepchem.feat.ConvMolFeaturizer()
x = featurizer.featurize(mols)

predicted_solubility = graph_conv.predict_on_batch(x)
predicted_solubility

array([[-0.29468927],
       [-0.05113898],
       [-0.06638844],
       [ 0.06888281],
       [ 0.02574509],
       [ 0.07853852],
       [-0.4667117 ],
       [-0.16995768],
       [-0.09682725],
       [-0.43664733],
       [-0.2026019 ],
       [-0.2934114 ]], dtype=float32)